# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("weather_data.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City,Longitude,Latitude,Temperature,Humidity,Wind Speed,Cloud Coverage,Country,Date
0,port-aux-francais,70.2167,-49.3500,6.53,87,16.05,99,TF,1689686393
1,talnakh,88.3972,69.4865,12.87,86,5.86,100,RU,1689686343
2,bodaybo,114.1933,57.8506,14.63,74,1.20,100,RU,1689686394
3,chonchi,-73.8092,-42.6128,2.29,83,1.57,100,CL,1689686394
4,markala,-6.0659,13.7021,36.04,34,2.92,93,ML,1689686395


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
import pandas as pd
import requests
import folium
from pathlib import Path

# Import API key
from api_keys import geoapify_key
geoapify_key = "b08a1d7fa92343428be395438e6324b3"

# Function to get the latitude and longitude for a city using Geoapify API
def get_coordinates(city, api_key):
    base_url = "https://api.geoapify.com/v1/geocode/search"
    params = {
        "text": city,
        "apiKey": api_key
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    if data["features"]:
        latitude = data["features"][0]["properties"]["lat"]
        longitude = data["features"][0]["properties"]["lon"]
        return latitude, longitude
    else:
        return None, None

# Load the weather data from CSV
df = pd.read_csv('weather_data.csv')

# Get latitude and longitude for each city and store them in the DataFrame
df['Latitude'], df['Longitude'] = zip(*df['City'].apply(lambda x: get_coordinates(x, geoapify_key)))

# Drop rows where latitude or longitude is missing
df = df.dropna(subset=['Latitude', 'Longitude'])

# Create a folium map centered at the mean latitude and longitude
map_weather = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=2)

# Add markers for each city with humidity as the size of points
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=row['Humidity'] * 0.1,  # Adjust the scaling factor as needed
        popup=row['City'],
        fill=True,
        fill_opacity=0.7,
        color='blue',
        fill_color='blue'
    ).add_to(map_weather)

# Display the map
map_weather


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values

# Assuming you have already loaded the weather data into the DataFrame 'df'

# Filter the DataFrame to include rows with temperature in the range of 20.0 to 26.0 degrees
filtered_df = df[(df['Temperature'] >= -3.0) & (df['Temperature'] <= 26.0)]

# Display the filtered DataFrame
print()


# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
filtered_df.head()

,City,Longitude,Latitude,Temperature,Humidity,Wind Speed,Cloud Coverage,Country,Date
0,port-aux-francais,70.219956,-49.349761,6.53,87,16.05,99,TF,1689686393
1,talnakh,88.390761,69.493989,12.87,86,5.86,100,RU,1689686343
2,bodaybo,114.203997,57.849953,14.63,74,1.20,100,RU,1689686394
3,chonchi,-73.772442,-42.623976,2.29,83,1.57,100,CL,1689686394
5,hasaki,140.820132,35.752971,24.22,84,3.66,100,JP,1689686396


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
import pandas as pd

# Assuming you already have the 'filtered_df' DataFrame with the required columns (City, Country, Latitude, Longitude, and Humidity)
columns_to_copy = ['City', 'Country', 'Latitude', 'Longitude', 'Humidity']
# Create a copy of the filtered DataFrame
hotel_df = filtered_df[columns_to_copy].copy()

# Add an empty column named "Hotel Name" to the DataFrame
hotel_df ['Hotel Name'] = ""

# Display sample data from the 'hotel_df'
hotel_df.head()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

,City,Country,Latitude,Longitude,Humidity,Hotel Name
0,port-aux-francais,TF,-49.349761,70.219956,87,
1,talnakh,RU,69.493989,88.390761,86,
2,bodaybo,RU,57.849953,114.203997,74,
3,chonchi,CL,-42.623976,-73.772442,83,
5,hasaki,JP,35.752971,140.820132,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
import pandas as pd
import requests

# Set parameters to search for a hotel
radius = 10000
params = {
    "apikey": "b08a1d7fa92343428be395438e6324b3",
    "filter": "category=accommodation",
    "bias": "proximity"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    data = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_name = data["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
port-aux-francais - nearest hotel: No hotel found
talnakh - nearest hotel: No hotel found
bodaybo - nearest hotel: No hotel found
chonchi - nearest hotel: No hotel found
hasaki - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
udachny - nearest hotel: No hotel found
stanley - nearest hotel: No hotel found
st. john's - nearest hotel: No hotel found
puerto deseado - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
college - nearest hotel: No hotel found
nuuk - nearest hotel: No hotel found
santa elena de uairen - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
vorgashor - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
chernyshevskiy - nearest hotel: No hotel found
greymouth - nearest hotel: No hotel found
sitka - nearest hotel: No hotel found
port lincoln - nearest hotel: No hotel found
badger - 

mount gambier - nearest hotel: No hotel found
gaiman - nearest hotel: No hotel found
el granada - nearest hotel: No hotel found
lerwick - nearest hotel: No hotel found
lagoa - nearest hotel: No hotel found
el alto - nearest hotel: No hotel found
eduardo castex - nearest hotel: No hotel found
whakatane - nearest hotel: No hotel found
maffra - nearest hotel: No hotel found
kone - nearest hotel: No hotel found
fort st. john - nearest hotel: No hotel found
san mariano - nearest hotel: No hotel found
pacocha - nearest hotel: No hotel found
sao joao da barra - nearest hotel: No hotel found
la plata - nearest hotel: No hotel found
ambovombe - nearest hotel: No hotel found
inongo - nearest hotel: No hotel found
monteagudo - nearest hotel: No hotel found
oppdal - nearest hotel: No hotel found
kargasok - nearest hotel: No hotel found
skovorodino - nearest hotel: No hotel found
tocopilla - nearest hotel: No hotel found
ola - nearest hotel: No hotel found
lata - nearest hotel: No hotel found
worla

,City,Country,Latitude,Longitude,Humidity,Hotel Name
0,port-aux-francais,TF,-49.349761,70.219956,87,No hotel found
1,talnakh,RU,69.493989,88.390761,86,No hotel found
2,bodaybo,RU,57.849953,114.203997,74,No hotel found
3,chonchi,CL,-42.623976,-73.772442,83,No hotel found
5,hasaki,JP,35.752971,140.820132,84,No hotel found


In [6]:
# Set parameters to search for a hotel
radius = 10000# YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

In [19]:

# Create a folium map centered at the mean latitude and longitude
map_weather = folium.Map(location=[hotel_df['Latitude'].mean(), hotel_df['Longitude'].mean()], zoom_start=2)

# Add markers for each city with humidity as the size of points and custom popup content
for _, row in hotel_df.iterrows():
    popup_content = f"City: {row['City']}<br>Country: {row['Country']}<br>Hotel: {row['Hotel Name']}"
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=row['Humidity'] * 0.1,  # Adjust the scaling factor as needed
        popup=popup_content,
        fill=True,
        fill_opacity=0.7,
        color='blue',
        fill_color='blue'
    ).add_to(map_weather)

# Display the map
map_weather
